# CNN#

**Проверка качества классификатора**

In [ ]:
from nltk.stem.snowball import SnowballStemmer
import csv
import re
from nltk.corpus import stopwords
import pymorphy2
from nltk.stem.porter import *
from sklearn.metrics import classification_report
import numpy as np
import pickle 

stops = set(stopwords.words("english")) | set(stopwords.words("russian"))

morph=pymorphy2.MorphAnalyzer()
stemmer=SnowballStemmer('russian')


In [ ]:
def treatment_text(review):
    review_text = re.sub("[^а-яА-Яa-zA-Z0-9]", " ", review)
    words = review_text.lower().split()
    words = [w for w in words if not w in stops]
    words = [morph.parse(w)[0].normal_form for w in words]
    words = [stemmer.stem(w) for w in words]
    words = [w for w in words if not w in stops]
    return(words)

In [ ]:
# создание dataset/x-статья,y-tag(0,1)
# dataset = [sentence_origin,sentence_tr,tag,kws,[kw_in_sentence]]
dataset = []
file='articles.csv'

with open(file,'r', encoding='utf-8',newline='') as f:
    reader = csv.reader(f,delimiter=',')
    for row in reader:            
        if row!=[]:
            kws=''.join(row[0])
            text = ''.join(row[2])
            sentences_origin = text.split('.')
            sentences_tr = list(map(treatment_text,sentences_origin))
            kws_tr = treatment_text(kws)  
            kws_tr_set=set(kws_tr)  
            for sentence_tr,sentence_or in zip(sentences_tr,sentences_origin):
                sentence_tr_set=set(sentence_tr)
                if len(sentence_tr)>5 and kws_tr_set.intersection(sentence_tr_set):
                    dataset.append((sentence_or,' '.join(sentence_tr),'1',kws,' '.join(kws_tr_set.intersection(sentence_tr_set))))
                elif len(sentence_tr)>5:
                    dataset.append((sentence_or,' '.join(sentence_tr),'0',kws,'None'))


In [ ]:
with open('cnn_model.pickle', 'rb') as f:
    cls = pickle.load(f)

**Проверка качества выделения КС**

In [217]:
from nltk.stem.snowball import SnowballStemmer
import csv
import re
import pymorphy2
from nltk.stem.porter import *
import pickle
from deeppavlov.dataset_readers.basic_classification_reader import BasicClassificationDatasetReader
from deeppavlov.dataset_iterators.basic_classification_iterator import BasicClassificationDatasetIterator
from deeppavlov.dataset_iterators.basic_classification_iterator import BasicClassificationDatasetIterator
from deeppavlov.models.tokenizers.nltk_moses_tokenizer import NLTKMosesTokenizer
from deeppavlov.core.data.simple_vocab import SimpleVocabulary
from deeppavlov.models.sklearn import SklearnComponent
from deeppavlov.metrics.accuracy import sets_accuracy
import numpy as np

from nltk.corpus import stopwords
from google.oauth2 import service_account

# credentials = service_account.Credentials.from_service_account_file('./diplom-275218-a097a7baee3f.json')
# project_id = 'diplom-275218'
# private_key = 'diplom-275218-a097a7baee3f.json'
stops = set(stopwords.words("english")) | set(stopwords.words("russian"))
stops.add('рис')
stops.add('университет')
stops.add('брянск')

morph=pymorphy2.MorphAnalyzer()
stemmer=SnowballStemmer('russian')


dict_stop=set(['метод','определение','условие','момент','значение','результат','критерий',
               'работа','вариант','брянский государственный университет','научнотехнический вестник',
              'соответствие','такой образ','весь критерий','пример','выбор','ключевое слово','период',
              'уравнение','формула','множитель','повышение','оценка','проведение',
              'машина','нагрузка','брянская область','точка','случай','расчет','таблица','расчёт',
              'с показатель','град','обработка','статья','элемент','раз','применение','центр','форма','важная задача'])


with open('./cls_model.pickle', 'rb') as f:
    cls = pickle.load(f)

with open('./tf_idf_model.pickle', 'rb') as f:
    tfidf = pickle.load(f)

def treatment_text2(text):
    text = re.sub("[^а-яА-Яa-zA-Z0-9,.?]", " ", str(text))
    text = text.replace('\t',' ')
    text = text.replace('\n', ' ')
    while text.find('  ')!=-1:
        text = text.replace('  ',' ')
    text = str(text)
    return text

def treatment_text(review):
    review_text = re.sub("[^а-яА-ЯЁёa-zA-Z0-9]", " ", review)
    review_text = review_text.replace('ё','е')
    review_text = review_text.replace('Ё', 'Е')
    words = review_text.lower().split()
    words = [w for w in words if not w in stops]
    words = [morph.parse(w)[0].normal_form for w in words]
    words = [stemmer.stem(w) for w in words]
    words = [w for w in words if not w in stops]
    return(words)


def normalize(keyphrase):
    try:
        words = keyphrase.split()
        if len(words)>1:
            main_noun = words[-1]
            flag_noun = False
            flag_adj = False
            for i in words[:-1]:
                if 'NOUN' not in  morph.parse(i)[0].tag and 'ADJF' not in morph.parse(i)[0].tag:
                    return morph.parse(words[0])[0].lexeme[0][0] + ','+morph.parse(words[2])[0].lexeme[0][0]
                if 'NOUN' in morph.parse(i)[0].tag:
                    flag_noun=True
                if 'ADJF' in morph.parse(i)[0].tag:
                    flag_adj=True
            s = ''

            if flag_noun is False:
                for word in words[:-1]:
                    for i in morph.parse(word)[0].lexeme:
                        if morph.parse(main_noun)[0].tag.gender in i.tag and 'nomn' in i.tag:

                            s+=i.word + ' '
                            break
                return s + morph.parse(main_noun)[0].lexeme[0][0]

            elif flag_noun is True and flag_adj is False:
                s = morph.parse(words[0])[0].lexeme[0][0]
                for i in words[1:]:
                    try:
                        s+= ' '+morph.parse(i)[0].lexeme[1][0]
                    except:
                        pass
                return s

            else:
                flag_first_noun=False

                for word in words:
                    if 'ADJF' in  morph.parse(word)[0].tag and flag_first_noun is False:
                        rod = find_rod(words[words.index(word)+1:])
                        for i in morph.parse(word)[0].lexeme:
                            if rod in i.tag and 'nomn' in i.tag:

                                s+=i.word + ' '
                                break
                    elif 'ADJF' in  morph.parse(word)[0].tag and flag_first_noun is True:
                        rod = find_rod(words[words.index(word)+1:])
                        for i in morph.parse(word)[0].lexeme:
                            if rod in i.tag and 'gent' in i.tag:
                                s+=i.word + ' '
                                break
                    elif  'NOUN' in  morph.parse(word)[0].tag and flag_first_noun is False:
                        flag_first_noun=True

                        s += morph.parse(word)[0].lexeme[0][0] + ' '
                    else:

                        s += morph.parse(word)[0].lexeme[2][0] +' '


            return s
        elif len(words)==1:
            return morph.parse(words[0])[0].lexeme[0][0]
    except:
        pass
#         print(' '.join(words),'НЕ МОЖЕТ БЫТЬ ПРЕОБРАЗОВАН')

def find_rod(words):
    for w in words:
        if 'NOUN' in morph.parse(w)[0].tag:
            return morph.parse(w)[0].tag.gender


def find_adj(sentence, index_noun, pred_noun):
    if pred_noun == 0 and index_noun == 0:
        return 0
    if pred_noun == 0 and index_noun != 0:
        i = index_noun - 1
        flag_ad = False
        flag_noun = False
        while i >= 0:
            if 'ADJF' in morph.parse(sentence[i])[0].tag:
                flag_ad = True
            elif 'NOUN' in morph.parse(sentence[i])[0].tag:
                flag_noun = True
            elif flag_ad is False and flag_noun is False:
                return i + 1

            flag_ad = False
            flag_noun = False
            i -= 1
        return i + 1

    else:
        i = index_noun - 1
        flag_ad = False
        flag_noun = False
        while i > pred_noun:
            if 'ADJF' in morph.parse(sentence[i])[0].tag:
                flag_ad = True
            elif 'NOUN' in morph.parse(sentence[i])[0].tag:
                flag_noun = True
            elif flag_ad is False and flag_noun is False:
                return i + 1

            flag_ad = False
            flag_noun = False
            i -= 1
        return i + 1


def keyword_extraction(text):
    sentences_origin = text.split('.')
    sentences_tr = list(map(treatment_text, sentences_origin))
    test=[]
    for sentence_tr, sentence_or in zip(sentences_tr, sentences_origin):
        # sentence_tr_set = set(sentence_tr)
        # if len(sentence_tr) > 5 and kws_tr_set.intersection(sentence_tr_set):
        #     test.append((sentence_or, ' '.join(sentence_tr)))
        if len(sentence_tr) > 5:
            test.append((sentence_or, ' '.join(sentence_tr)))
    x_test = []
    for s in test:
        x_test.append(s[1])
#     x_test = tuple(x_test)
#     y_pred = model.predict(tokenizer.texts_to_matrix(x_test, mode='binary'))
#     print(y_pred,x_test)
    x_test = [i.split(' ')[:20] for i in x_test]
    vecX = vectorize(x_test, sentence_maxlen, w2i)
    result =  model.predict(vecX)
    
    y_pred2 = []
    for i in result:
        if i[0]>=0.5:
            y_pred2.append('1')
        else:
            y_pred2.append('0')
    

    sentences_with_kw = []
    for i in range(len(y_pred2)):
        if y_pred2[i] == '1':
#             print(test[i])
            sentences_with_kw.append(test[i])
    dict_cand = {}
    for sentence in sentences_with_kw:
        cand = extract_candidate(sentence[0], sentences_with_kw)
        cand = list(map(normalize, cand))
        for key_cand in cand:
            if key_cand in dict_cand:
                dict_cand[key_cand] += 1
            else:
                dict_cand[key_cand] = 1
    dict_cand_sort = {k: v for k, v in sorted(dict_cand.items(), key=lambda item: item[1], reverse=True)}
    result = []
    count = 0
    k = set(['1', '2', '3', '4', '5', '&', '?', '/', '\\', '!'])

    for word in dict_cand_sort.keys():
        try:
            if count < 10:
                if ',' in word:
                    word = word.split()
                    result += word
                elif k.intersection(set(word)) == set() and word not in dict_stop and 'ADJF' not in morph.parse(word)[
                    0].tag:
                    result.append(word)
                    count += 1
            else:
                break
        except:
            pass
    for i in result:

        flag = False
        s = i.replace(',', ' ')
        s = s.split(' ')
        for j in s:
            if 'NOUN' in morph.parse(j)[0].tag:
                flag = True
        if flag is False:
            result.remove(i)
    result = list(map(lambda x: x.replace(',', ' '), result))
    result = ', '.join(result)
    if result!='' :
        return result
#     else:
#         keywords = r.keywords_extract(text)
#         keywords = ', '.join(keywords).capitalize()
#         return keywords
def extract_candidate(origin, sentences_with_kw):
    candidate = []
    x = re.sub("[^а-яА-Я]", " ", origin)
    while x.find('  ') != -1:
        x = x.replace('  ', ' ')
    x = x.split()
    x = list(filter(lambda x: len(x) > 2, x))
    pr = 0

    for i in range(len(x) - 1):
        if 'NOUN' in morph.parse(x[i])[0].tag and 'NOUN' in morph.parse(x[i + 1])[0].tag and i + 2 != len(x):
            pass
        elif 'NOUN' in morph.parse(x[i])[0].tag and 'NOUN' in morph.parse(x[i + 1])[0].tag and i + 2 == len(x):
            try:
                k = find_adj(x, i + 2, pr)
                pr = i
                if ' '.join(x[k:i + 1]) in sentences_with_kw[2][0]:
                    candidate.append(' '.join(x[k:i + 2]))
                else:
                    begin = origin.find(x[k])
                    end = origin.find(x[i + 1]) + len(x[i + 1])
                    candidate += origin[begin:end].split(',')
            except:
                pass
        elif 'NOUN' in morph.parse(x[i])[0].tag and 'NOUN' not in morph.parse(x[i + 1])[0].tag:

            k = find_adj(x, i, pr)
            pr = i

            if ' '.join(x[k:i + 1]) in origin:
                candidate.append((' '.join(x[k:i + 1])))
            else:
                begin = origin.find(x[k], len(' '.join(x[:k])))
                end = origin.find(x[i], len(' '.join(x[:i]))) + len(x[i])
                candidate += origin[begin:end].split(',')
    return candidate

In [219]:
# создание dataset/x-статья,y-tag(0,1)
# dataset = [sentence_origin,sentence_tr,tag,kws,[kw_in_sentence],text]
result = []
file='test_sample_kws.csv'
k=0

with open(file,'r', encoding='cp1251',newline='') as f:
    reader = csv.reader(f,delimiter=';')
    for row in reader:
        if row!=[]:
            print(k)
            k+=1
            kws_true =''.join(row[0])
            ann = ''.join(row[1])
            text = ''.join(row[2])
            if text!='':
                kws_pred = keyword_extraction(text)
                result.append([kws_true,kws_pred])
            
                
                

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63


In [220]:
len_true = []
len_pred = []
len_intersection = []

for i in range(len(result)-1):
    kws_true = set(treatment_text(result[i][0])) 
    kws_pred = set(treatment_text(result[i][1])) 
    len_true.append(len(kws_true))
    len_pred.append(len(kws_pred))
    len_intersection.append(len(kws_true.intersection(kws_pred)))
#     print(result[i][0])
#     print(result[i][1])
#     print(len(kws_true.intersection(kws_pred)))


recall =sum(len_intersection)/sum(len_true)
precision = sum(len_intersection)/sum(len_pred)
print(f'precision    {precision}')
print(f'recall       {recall}')
print(f'f_measure    {2*precision*recall/(precision+recall)}')

precision    0.3144758735440932
recall       0.4643734643734644
f_measure    0.375


**Обучение модели**

In [5]:
# создание dataset/x-статья,y-tag(0,1)
# dataset = [sentence_origin,sentence_tr,tag,kws,[kw_in_sentence]]
dataset = []

files=['./train_log/articles2019.csv','./train_log/articles2018.csv','./train_log/articles2017.csv']
for file in files:
    with open(file,'r', encoding='utf-8',newline='') as f:
        reader = csv.reader(f,delimiter=',')
        for row in reader:     
            if row!=[]:
                kws=''.join(row[0])
                text = ''.join(row[2])
                sentences_origin = text.split('.')
                sentences_tr = list(map(treatment_text,sentences_origin))
                kws_tr = treatment_text(kws)  
                kws_tr_set=set(kws_tr)        
                for sentence_tr,sentence_or in zip(sentences_tr,sentences_origin):
                    sentence_tr_set=set(sentence_tr)
                    if len(sentence_tr)>5 and kws_tr_set.intersection(sentence_tr_set):
                        dataset.append((sentence_or,' '.join(sentence_tr),'1',kws,' '.join(kws_tr_set.intersection(sentence_tr_set))))
                    elif len(sentence_tr)>5:
                        dataset.append((sentence_or,' '.join(sentence_tr),'0',kws,'None'))



array([[  616, 10771,   678, ...,     0,     0,     0],
       [ 1129,  8619, 12440, ...,     0,     0,     0],
       [ 1358,  6418,  9357, ...,     0,     0,     0],
       ...,
       [10798, 10957, 13584, ...,     0,     0,     0],
       [ 1358, 10011, 11226, ...,     0,     0,     0],
       [ 7213,  8146,  9532, ...,     0,     0,     0]])

In [4]:
import numpy as np
import codecs
import os
import random

from keras import backend as K
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Dense, Lambda, Permute, Dropout
from keras.layers import Conv2D, MaxPooling1D

In [195]:
# def load_data(fpath, label):
#     data = []
#     with codecs.open(fpath, 'r', 'utf-8', errors='ignore') as f:
#         lines = f.readlines()
#         for l in lines:
#             l = l.rstrip()
#             data.append((l.split(' '), label))
#     return data
# pos = load_data('./dataset/rt-polaritydata/rt-polarity.pos', 1)
# neg = load_data('./dataset/rt-polaritydata/rt-polarity.neg', 0)
# data = pos + neg

x = [dataset[i][1].split() for i in range(len(dataset))]
y = [dataset[i][2] for i in range(len(dataset))]

In [196]:
sentence_maxlen = max(map(len, (d for d in x)))
print('sentence maxlen', sentence_maxlen)
sentence_maxlen=20
x = [i[:sentence_maxlen] for i in x]

sentence maxlen 407


In [197]:
vocab = []
k=0
for d in x:
    k+=1
    if k % 1000==0 :
        print(k)
    for w in d:
        if w not in vocab: vocab.append(w)
vocab = sorted(vocab)
vocab_size = len(vocab)
print('vocab examples:', vocab[:10])

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
vocab examples: ['0', '00', '000', '0000000225676336', '0000000234101139', '00001', '0000зуx', '00017', '000244', '00034']


In [198]:
len(x)

12668

In [199]:
print('vocab size', len(vocab))
w2i = {w:i for i,w in enumerate(vocab)}
# i2w = {i:w for i,w in enumerate(vocab)}
# w2i['character']

vocab size 14097


In [200]:
def vectorize(data, sentence_maxlen, w2i):
    vec_data = []
    for d in data:
        vec = [w2i[w] for w in d if w in w2i]
        pad_len = max(0, sentence_maxlen - len(vec))
        vec += [0] * pad_len
        vec_data.append(vec)
    vec_data = np.array(vec_data)
    
    return vec_data
# random.shuffle(data)

vecX = vectorize(x, sentence_maxlen, w2i)
vecY=np.array(y)

In [201]:
n_data = len(vecX)
split_ind = (int)(n_data * 0.9)
trainX, trainY = vecX[:split_ind], vecY[:split_ind]
testX, testY = vecX[split_ind:], vecY[split_ind:]

In [202]:

def load_glove_weights(glove_dir, embd_dim, vocab_size, word_index):
    embeddings_index = {}
    f = open(os.path.join(glove_dir, 'glove.6B.' + str(embd_dim) + 'd.txt'),encoding='utf-8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

    print('Found %s word vectors.' % len(embeddings_index)) 
    embedding_matrix = np.zeros((vocab_size, embd_dim))
    print('embed_matrix.shape', embedding_matrix.shape)
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

    return embedding_matrix

embd_dim = 100
glove_embd_w = load_glove_weights(r'C:\Users\Ирина\PycharmProjects\UIR7sem\venv\\', embd_dim, vocab_size, w2i)

Found 400001 word vectors.
embed_matrix.shape (14097, 100)


In [203]:
# from deeppavlov.models.embedders.glove_embedder import GloVeEmbedder
# from deeppavlov.core.data.utils import simple_download
# # simple_download(url="http://files.deeppavlov.ai/embeddings/glove.6B.100d.txt", 
# #                 destination="./glove.6B.100d.txt")
# embedder = GloVeEmbedder(load_path=r'C:\Users\Ирина\PycharmProjects\UIR7sem\venv\glove.6B.100d.txt',
#                          dim=100, pad_zero=True)
# # output shape is (batch_size x max_num_tokens_in_the_batch x embedding_dim)
# # embedded_batch = embedder(str_lower(tokenizer(['Is it freezing in Offerman, California?']))) 
# # len(embedded_batch), len(embedded_batch[0]), embedded_batch[0][0].shape

In [204]:
def Net(vocab_size, embd_size, sentence_maxlen, glove_embd_w):
    sentence = Input((sentence_maxlen,), name='SentenceInput')
    
    # embedding
    embd_layer = Embedding(input_dim=vocab_size, 
                           output_dim=embd_size, 
                           weights=[glove_embd_w], 
                           trainable=False,
                           name='shared_embd')
    embd_sentence = embd_layer(sentence)
    embd_sentence = Permute((2,1))(embd_sentence)
    embd_sentence = Lambda(lambda x: K.expand_dims(x, -1))(embd_sentence)
    
    # cnn
    cnn = Conv2D(1, 
                 kernel_size=(3, sentence_maxlen),
                 activation='relu')(embd_sentence)
    cnn =  Lambda(lambda x: K.sum(x, axis=3))(cnn)
    cnn = MaxPooling1D(3)(cnn)
    cnn = Lambda(lambda x: K.sum(x, axis=2))(cnn)
    out = Dense(1, activation='sigmoid')(cnn)

    model = Model(inputs=sentence, outputs=out, name='sentence_claccification')
    model.compile(optimizer='adagrad', loss='binary_crossentropy', metrics=['accuracy']) 
    return model

model = Net(vocab_size, embd_dim, sentence_maxlen, glove_embd_w)
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
SentenceInput (InputLayer)   (None, 20)                0         
_________________________________________________________________
shared_embd (Embedding)      (None, 20, 100)           1409700   
_________________________________________________________________
permute_12 (Permute)         (None, 100, 20)           0         
_________________________________________________________________
lambda_34 (Lambda)           (None, 100, 20, 1)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 98, 1, 1)          61        
_________________________________________________________________
lambda_35 (Lambda)           (None, 98, 1)             0         
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 32, 1)             0         
__________

In [205]:
model.fit(trainX, trainY,
            batch_size=32,
            epochs=10,
            validation_data=(testX, testY)
        )

Train on 11401 samples, validate on 1267 samples
Epoch 1/10
11401/11401 [==============================] - ETA: 10:44 - loss: 0.8513 - acc: 0.34 - ETA: 2:45 - loss: 0.7369 - acc: 0.4688 - ETA: 1:52 - loss: 0.7184 - acc: 0.505 - ETA: 1:26 - loss: 0.7077 - acc: 0.515 - ETA: 1:10 - loss: 0.6995 - acc: 0.534 - ETA: 1:00 - loss: 0.6945 - acc: 0.528 - ETA: 53s - loss: 0.6915 - acc: 0.522 - ETA: 47s - loss: 0.6897 - acc: 0.52 - ETA: 40s - loss: 0.6835 - acc: 0.53 - ETA: 37s - loss: 0.6809 - acc: 0.53 - ETA: 33s - loss: 0.6823 - acc: 0.52 - ETA: 30s - loss: 0.6821 - acc: 0.52 - ETA: 28s - loss: 0.6826 - acc: 0.52 - ETA: 26s - loss: 0.6793 - acc: 0.52 - ETA: 24s - loss: 0.6786 - acc: 0.52 - ETA: 23s - loss: 0.6758 - acc: 0.53 - ETA: 21s - loss: 0.6758 - acc: 0.53 - ETA: 20s - loss: 0.6716 - acc: 0.54 - ETA: 19s - loss: 0.6732 - acc: 0.54 - ETA: 18s - loss: 0.6706 - acc: 0.54 - ETA: 18s - loss: 0.6690 - acc: 0.54 - ETA: 17s - loss: 0.6669 - acc: 0.55 - ETA: 16s - loss: 0.6668 - acc: 0.55 - ETA: 

Epoch 4/10
11401/11401 [==============================] - ETA: 2s - loss: 0.5566 - acc: 0.750 - ETA: 2s - loss: 0.6053 - acc: 0.673 - ETA: 2s - loss: 0.6215 - acc: 0.674 - ETA: 2s - loss: 0.6247 - acc: 0.661 - ETA: 2s - loss: 0.6241 - acc: 0.655 - ETA: 2s - loss: 0.6204 - acc: 0.665 - ETA: 2s - loss: 0.6242 - acc: 0.660 - ETA: 2s - loss: 0.6222 - acc: 0.656 - ETA: 2s - loss: 0.6221 - acc: 0.653 - ETA: 2s - loss: 0.6267 - acc: 0.649 - ETA: 2s - loss: 0.6252 - acc: 0.651 - ETA: 1s - loss: 0.6290 - acc: 0.646 - ETA: 1s - loss: 0.6302 - acc: 0.644 - ETA: 1s - loss: 0.6285 - acc: 0.643 - ETA: 1s - loss: 0.6287 - acc: 0.641 - ETA: 1s - loss: 0.6282 - acc: 0.641 - ETA: 1s - loss: 0.6290 - acc: 0.640 - ETA: 1s - loss: 0.6277 - acc: 0.639 - ETA: 1s - loss: 0.6267 - acc: 0.638 - ETA: 1s - loss: 0.6265 - acc: 0.639 - ETA: 1s - loss: 0.6269 - acc: 0.639 - ETA: 1s - loss: 0.6262 - acc: 0.640 - ETA: 1s - loss: 0.6249 - acc: 0.644 - ETA: 1s - loss: 0.6243 - acc: 0.645 - ETA: 1s - loss: 0.6239 - acc: 

11401/11401 [==============================] - ETA: 2s - loss: 0.6820 - acc: 0.593 - ETA: 2s - loss: 0.6237 - acc: 0.656 - ETA: 2s - loss: 0.6236 - acc: 0.651 - ETA: 2s - loss: 0.6149 - acc: 0.659 - ETA: 1s - loss: 0.6168 - acc: 0.647 - ETA: 1s - loss: 0.6176 - acc: 0.646 - ETA: 1s - loss: 0.6124 - acc: 0.662 - ETA: 1s - loss: 0.6133 - acc: 0.659 - ETA: 1s - loss: 0.6135 - acc: 0.657 - ETA: 1s - loss: 0.6145 - acc: 0.654 - ETA: 1s - loss: 0.6129 - acc: 0.659 - ETA: 1s - loss: 0.6109 - acc: 0.659 - ETA: 1s - loss: 0.6105 - acc: 0.658 - ETA: 1s - loss: 0.6098 - acc: 0.659 - ETA: 1s - loss: 0.6104 - acc: 0.657 - ETA: 1s - loss: 0.6115 - acc: 0.656 - ETA: 1s - loss: 0.6117 - acc: 0.655 - ETA: 1s - loss: 0.6117 - acc: 0.654 - ETA: 1s - loss: 0.6112 - acc: 0.655 - ETA: 1s - loss: 0.6127 - acc: 0.654 - ETA: 1s - loss: 0.6134 - acc: 0.651 - ETA: 1s - loss: 0.6137 - acc: 0.651 - ETA: 1s - loss: 0.6143 - acc: 0.651 - ETA: 1s - loss: 0.6146 - acc: 0.651 - ETA: 0s - loss: 0.6144 - acc: 0.651 - ETA

In [206]:
# trainX = np.array([np.array(trainX[i][:100]).reshape(100,) for i in range(len(trainX))])
# testX = np.array([np.array(testX[i][:100]).reshape(100,) for i in range(len(testX))])

In [207]:
# d=set()
# for i in trainX:
#     d.add(len(i))


In [208]:

pred = []
for i in model.predict(testX):
    
    if i[0]>=0.5:
        pred.append(1)
    else:
         pred.append(0)



In [209]:
y_true = list(map(lambda x: int(x[0]),testY))
from sklearn.metrics import classification_report
print(classification_report(y_true, pred, labels=[0, 1]))


              precision    recall  f1-score   support

           0       0.67      0.76      0.71       614
           1       0.74      0.66      0.70       653

    accuracy                           0.71      1267
   macro avg       0.71      0.71      0.71      1267
weighted avg       0.71      0.71      0.71      1267



In [138]:
30
precision    recall  f1-score   support

           0       0.67      0.74      0.70       614
           1       0.73      0.66      0.69       653

    accuracy                           0.70      1267
   macro avg       0.70      0.70      0.70      1267
weighted avg       0.70      0.70      0.70      1267


In [ ]:
50
              precision    recall  f1-score   support

           0       0.68      0.74      0.71       614
           1       0.74      0.68      0.71       653

    accuracy                           0.71      1267
   macro avg       0.71      0.71      0.71      1267
weighted avg       0.71      0.71      0.71      1267

In [ ]:
20 

              precision    recall  f1-score   support

           0       0.67      0.76      0.71       614
           1       0.74      0.66      0.70       653

    accuracy                           0.71      1267
   macro avg       0.71      0.71      0.71      1267
weighted avg       0.71      0.71      0.71      1267